In [ ]:
import pandas as pd
import random
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from functools import partial
from sklearn import preprocessing
import tensorflow.keras.losses as losses
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import mean_absolute_error #MAE
# 定义超参数空间
hyperparam_space = {
    'batch_size': [16, 32],
    'learning_rate': [0.001, 0.01, 0.1],
    'num_filters': [8, 16, 32, 64, 128],
    'kernel_size': [3, 5, 7, 10, 15, 16, 20],
    'dense_units': [16, 32, 64, 128],
    'pool_size':[2,3,4],#池化层窗口数量
    'sigma':[0.1,0.3,0.5,0.7]

}
kh_r2 = []
kh_mse = []
kh_rmse = []
kh_mae = []
kh_rpd = []
kh_mape = []
indexs = []
kh_mspe = []
kh_smape = []
# 遗传算法主函数
curves = []
curves_2 = []
rmsep = []
rmsec = []
def genetic_algorithm(x_train, y_train, x_test, y_test, pop_size, num_generations):
    # 初始化种群
    crossover_prob_max = 0.8
    crossover_prob_min = 0.05
    crossover_prob = 0.4
    mutation_prob = 0.1
    mutation_prob_max = 0.1
    mutation_prob_min = 0.01
    def init_population(pop_size):
        population = []
        for i in range(pop_size):
            hyperparams = {}
            for param_name, param_values in hyperparam_space.items():
                hyperparams[param_name] = random.choice(param_values)
            population.append(hyperparams)
        return population

    # 计算损失函数
    def kernel_huber_loss(y_true, y_pred, kernel_fn, delta):
        # Compute the pairwise kernel values between labels
        K = kernel_fn(y_true, y_true)

        # Compute the residual error
        residual = y_true - y_pred

        # Apply the huber loss function
        mask = tf.abs(residual) < delta
        huber_loss = tf.where(mask, 0.5 * tf.square(residual), delta * (tf.abs(residual) - 0.5 * delta))

        # Compute the final loss
        loss = tf.reduce_sum(K * huber_loss) / tf.reduce_sum(K)

        return loss

    def gaussian_kernel(X1, X2, sigma=1.0):
        pairwise_sq_dists = tf.reduce_sum(tf.square(X1), axis=[1], keepdims=True) + \
                            tf.reduce_sum(tf.square(X2), axis=[1]) - \
                            2.0 * tf.matmul(X1, X2, transpose_b=True)
        return tf.exp(-pairwise_sq_dists / (2.0 * sigma ** 2))
    def MSPE(true, pred):
        true = true + 0.1  # 避免0值
        diff = np.array(true) - np.array(pred)
        return np.mean(np.power(diff / true, 2))

    def mape(y_true, y_pred):

        return np.mean(np.abs((y_pred - y_true) / y_true)) * 100
    def smape(y_true, y_pred):
        return 2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true))) * 100



    # 构建模型
    def build_model(hyperparams):
        input_layer = tf.keras.layers.Input(shape=(8, 1))
        x = tf.keras.layers.Conv1D(hyperparams['num_filters'], hyperparams['kernel_size'], activation='relu', padding='same')(input_layer)
        residual = x
        x = tf.keras.layers.Conv1D(hyperparams['num_filters'], hyperparams['kernel_size'], activation='relu', padding='same')(x)
        x = tf.keras.layers.Conv1D(hyperparams['num_filters'], hyperparams['kernel_size'], activation='relu', padding='same')(x)
        x = tf.keras.layers.Add()([x, residual])
        x = tf.keras.layers.MaxPooling1D(pool_size=hyperparams['pool_size'], strides=hyperparams['pool_size'], padding='same')(x)
        x = tf.keras.layers.Flatten()(x)
        x = tf.keras.layers.Dense(hyperparams['dense_units'], activation='relu')(x)
        output_layer = tf.keras.layers.Dense(1, activation='linear')(x)
        model = tf.keras.Model(inputs=input_layer, outputs=output_layer)


        optimizer = tf.keras.optimizers.Adam(learning_rate=hyperparams['learning_rate'])
        kernel_fn = partial(gaussian_kernel, sigma=hyperparams['sigma'])
        model.compile(optimizer=optimizer, loss=lambda y_true, y_pred: kernel_huber_loss(y_true, tf.cast(y_pred, tf.float32), kernel_fn, delta=1.0))

        return model


    # 交叉操作
    # def crossover(parent1, parent2):
    #     child = {}
    #     for param_name, param_values in hyperparam_space.items():
    #         if random.random() < 0.5:
    #             child[param_name] = parent1[param_name]
    #         else:
    #             child[param_name] = parent2[param_name]
    #     return child

    def crossover(parent1, parent2):
        child = {}
        for param_name, param_values in hyperparam_space.items():
            if random.random() < crossover_prob:
                child[param_name] = parent1[param_name]
            else:
                child[param_name] = parent2[param_name]
        return child

    # 变异操作
    # def mutate(individual):
    #     mutated = {}
    #     for param_name, param_values in hyperparam_space.items():
    #         if random.random() < 0.2:
    #             mutated[param_name] = random.choice(param_values)
    #         else:
    #             mutated[param_name] = individual[param_name]
    #     return mutated
    def mutate(individual):
        mutated = {}
        for param_name, param_values in hyperparam_space.items():
            if random.random() < mutation_prob:
                mutated[param_name] = random.choice(param_values)
            else:
                mutated[param_name] = individual[param_name]
        return mutated

    # 评估适应度
    def evaluate_fitness(model, x, y):
        score = model.evaluate(x, y, verbose=0)
        return 1 / score

    # 初始化种群
    population = init_population(pop_size)
    for gen in range(num_generations):
        print(f"Generation {gen+1}/{num_generations}")
        # 训练和评估每个个体

        # # estimate the population fitness
        # fitness_scores = []
        # for i, individual in enumerate(population):
        #     model = build_model(individual)
        #     fitness_score = evaluate_fitness(model, x_train, y_train)
        #     fitness_scores.append((individual, fitness_score))
        # # sort the population by descending fitness
        # fitness_scores.sort(key=lambda x: x[1], reverse=True)
        # # calculate mean and max fitness
        # mean_fitness = sum([score for _, score in fitness_scores]) / len(population)
        # max_fitness = fitness_scores[0][1]
        # # adjust mutation and crossover probabilities based on fitness
        # mutation_prob *= (mean_fitness / max_fitness)
        # crossover_prob /= (mean_fitness / max_fitness)


        fitness_scores = []
        for i, individual in enumerate(population):
            model = build_model(individual)
            history = model.fit(x_train, y_train, batch_size=individual['batch_size'], epochs=50, verbose=0)

            y_pre = model.predict(x_test)
            if(r2_score(y_test,y_pre)>=0.9):
                y_test_np = np.array(y_test)
                kh_r2.append(r2_score(y_test,y_pre))
                mse = mean_squared_error(y_test, y_pre)
                kh_mse.append(mse)
                kh_rmse.append(sqrt(mse))
                kh_rpd.append(np.std(y_test) / sqrt(mse))
                kh_mae.append(mean_absolute_error(y_test,y_pre))
                kh_mape.append(mape(y_test,y_pre))
                indexs.append(i)
                kh_mspe.append(MSPE(y_test,y_pre))
                kh_smape.append(smape(y_test,y_pre))
                msep = mean_squared_error(y_train,model.predict(x_train))
                rmsec.append(sqrt(mse))
                rmsep.append(sqrt(msep))
            loss = history.history['loss'][-1]
            fitness_score = evaluate_fitness(model, x_test, y_test)
            fitness_scores.append((individual, fitness_score))
            # fitness_scores.sort(key=lambda x: x[1], reverse=True)
            print(f"Individual {i+1}/{pop_size}, Loss: {loss:.4f}, Fitness score: {fitness_score:.4f}")
        fitness_scores.sort(key=lambda x: x[1], reverse=True)
        # calculate mean and max fitness
        mean_fitness = sum([score for _, score in fitness_scores]) / len(population)
        max_fitness = fitness_scores[0][1]
        # adjust mutation and crossover probabilities based on fitness
        if(fitness_score >= mean_fitness ):
            mutation_prob = ((mutation_prob_max - mutation_prob_min)*(1 - ((gen + 1) / num_generations)))/(1 + np.exp(9.903*((2*(fitness_score - mean_fitness))/(max_fitness - mean_fitness))))
            # mutation_prob *= (mean_fitness / max_fitness)
            # crossover_prob /= (mean_fitness / max_fitness)
            crossover_prob = ((crossover_prob_max - crossover_prob_min)*(1 - ((gen + 1) / num_generations)))/(1 + np.exp(9.903*((2*(fitness_score - mean_fitness))/(max_fitness - mean_fitness))))
        else:
            mutation_prob *= (mean_fitness / max_fitness)
            crossover_prob /= (mean_fitness / max_fitness)


        # 排序并输出最佳个体
        best_individual, best_fitness = sorted(fitness_scores, key=lambda x: x[1], reverse=True)[0]
        curves.append(best_fitness)
        curves_2.append(mean_fitness)
        print(f"Best individual: {best_individual}, Best fitness: {best_fitness}")

        # 选择新一代的父代
        parents = []
        total_fitness = sum([score for _, score in fitness_scores])
        for i in range(pop_size):
            pick = random.uniform(0, total_fitness)
            current = 0
            for individual, fitness_score in fitness_scores:
                current += fitness_score
                if current > pick:
                    parents.append(individual)
                    break
          # 生成新一代个体
        new_population = []
        for i in range(pop_size):
            parent1 = random.choice(parents)
            parent2 = random.choice(parents)
            child = crossover(parent1, parent2)
            mutated_child = mutate(child)
            new_population.append(mutated_child)
        population = new_population
    return best_individual
def load_data():
    all_data = pd.read_csv('data.csv', encoding='GBK')
    all_data = np.array(all_data)
    scaler = preprocessing.StandardScaler().fit(all_data)
    all_data = scaler.transform(all_data)
    x_data = all_data[:,:-1]
    y_data = all_data[:,[-1]]
    x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.2, random_state=3051043)
    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = load_data()
best_individual = genetic_algorithm(x_train, y_train, x_test, y_test, pop_size=5, num_generations=20)
print(f"Best hyperparameters: {best_individual}")
list_all = np.stack(arrays=(indexs,kh_r2,kh_mse,kh_rmse,kh_rpd,kh_mae,kh_mape,kh_mspe,kh_smape,rmsec,rmsep), axis=0).tolist()

dicts = {}
for i in range(len(list_all)):
    dicts[i] = list_all[i]
df_data = pd.DataFrame(dicts)
df_data.columns = ['I','R^2','MSE','RMSE','RPD','MAE','MAPE','MSPE','SMAPE','RMSEC','RMSEP']
df_data.to_csv('mapes.csv')
curves = pd.DataFrame(curves)
curves.to_csv('improved_ga_cnn_max_curve.csv')
curves_2 = pd.DataFrame(curves_2)
curves_2.to_csv('improved_ga_cnn_mean_curve.csv')
print("++++++++++++++++DONE！！！+++++++++++++++++")
